# 0. **Start Here**

## **Jupyter Notebooks**
Jupyter Notebooks are interactive documents that allow users to run code \
in a web browser. Code is organized into small cells, so that users can \
easily see and interpret the output.

### Basic Operation
* **Run cell**: Shift + Enter
* **Change cell**: Left click in cell, make changes, press Enter
* **Restart notebook**: In the Jupyter browser window:
    + Left click the Kernel menu.
    + Left click the desired option.
        - Most users want Restart Kernel and Clear All Outputs. 
* **Save notebook**: Control + S
* **Shut down**: In the Jupyter browser window:
    + left click the File menu at upper-left.
    + left click Shut Down in the dropdown menu. 
    + left click Shut Down in the pop-up menu.

## **Choose a Julia kernel**
The upper-right corner of the tutorial window should show a Julia version \
string, like "Julia 1.5.4". \
\
If it says "No Kernel", left click that message, drag the pop-up menu \
selector to the desired Julia language version, then left click "Select".

## **Run Setup Commands**
If you haven't run this tutorial before as the current user, then please run \
this set of commands: (press Shift + Enter)

In [ ]:
using Pkg
Pkg.resolve()
if Sys.iswindows() == false
    if get(Pkg.installed(), "SeisIO", nothing) == nothing
        println("SeisIO not installed; calling Pkg.add.")
        # Pkg.add("SeisIO")
    else
        println("SeisIO installed.")
    end
else
    println("Windows detected; assuming SeisIO is installed.")
end

# 1. **Getting Started**
Start SeisIO by loading the package into memory:

In [ ]:
using SeisIO

You'll need to do this at the start of every Julia session. 
\
\
SeisIO uses an array-like structure called `SeisChannel` for single-channel data.

In [ ]:
C = SeisChannel()

\
\
Access fields by name to view their contents directly:

In [ ]:
C.x

\
You can also overwrite them by name:

In [ ]:
C.x = rand(Float32, 1024)

In [ ]:
C.loc = GeoLoc(lat = -90.0, lon = 0.0, el = 9300.0)

In [ ]:
C

\
The `SeisData` structure is the basic container that SeisIO uses. \
These objects behave like arrays of `SeisChannel` objects: they store multiple \
channels of data, but they're not matrices (more like arrays of arrays).\
You can initialize `SeisData` structures as empty containers for 0 or more channels:

In [ ]:
S = SeisData(6)

\
Let's create a random example for tutorial purposes.

In [ ]:
using SeisIO.RandSeis
S = randSeisData(6)

\
SeisData fields are accessed by name and index.

In [ ]:
x1 = S.x[1]        # get first index of data field :x

\
If you access a SeisData structure by a single integer index, the output is a SeisChannel:

In [ ]:
C_new = S[1]

\
If you access a SeisData structure by a range of indices, the output is a\
SeisData structure whose channel indices match your request:

In [ ]:
S2 = S[1:3]

\
These commands access identical information, but the first is faster:

In [ ]:
x1 = S.x[1]        # get first index of data field :x
x2 = S[1].x        # extract S[1] to a SeisChannel, then get its data field :x

In [ ]:
x1 == x2           # true unless there are NaNs

## Working with Structures (Optional Section)
A collection of SeisChannel objects becomes a SeisData structure:

In [ ]:
S = SeisData(randSeisChannel(), randSeisChannel())

\
You can push channels onto existing SeisData structures:

In [ ]:
push!(S, C_new)

\
SeisData structures can be concatenated:

In [ ]:
append!(S, randSeisData(4))
S

\
The addition operator also does this:

In [ ]:
S += randSeisChannel()

\
Thus, this command works: (note: please don't actually code like this)

In [ ]:
S = SeisData(
        randSeisData(5), 
        randSeisChannel(), 
        SeisChannel(
            id="UW.SEP..EHZ", 
            name="Darth Exploded", 
            loc=GeoLoc(lat=46.1967, lon=-122.1875, el=1440.0), 
            t=[1 1569374150379000; 1024 0], 
            x=rand(1024)
            )
        )

\
**Warning**: Combining structures calls `prune!` to remove empty channels,\
defined as channels with no data (empty `:x`); this is one of two operations\
needed to guarantee commutativity (i.e., for any `SeisData` structures S1, S2,\
`S1 + S2 == S2 + S1`). Thus, in the (heinous) creation syntax below, the command\
`SeisData(3)`, which should add three empty channels, does nothing; `prune!`\
deletes the new channels. (To actually append empty channels, use `append!`)

In [ ]:
S1 = SeisData(
        randSeisData(5), 
        randSeisChannel(), 
        SeisChannel(
            id="UW.SEP..EHZ", 
            name="Darth Exploded", 
            loc=GeoLoc(lat=46.1967, lon=-122.1875, el=1440.0), 
            t=[1 1569374150379000; 1024 0], 
            x=rand(1024)
            )
        ) + SeisData(3)

## Navigating Structures (Optional Section)
There are two easy ways to find channels of interest in a data structure.\
The command `findid` returns the first channel number that matches the ID\
supplied:

In [ ]:
i = findid("UW.SEP..EHZ", S)

\
If you want partial matches, use the function `findchan`:

In [ ]:
findchan("EHZ", S)

## Variant Structures (Optional Section)
`SeisData` is a subtype of `GphysData`. The Abstract Type `GphysData` \
includes variant data structures, all of which have the same basic fields\
(`:id`, `:x`, etc). Other data structures will be added to `GphysData`\
as modules develop, but they will always contain the same basic fields.\
For example, in the Quake submodule, the `EventTraceData` structure\
has additional fields for location parameters like azimuth (`:az`)\
and has a custom Type for a phase catalog (`:pha`).\
\
You can convert between `GphysData` subtypes with `convert`, but extraneous\
fields will be lost, and fields that are not contained in the source Type\
will be initialized to default values:

In [ ]:
using SeisIO.Quake
S_ev = randSeisData(6)
Tr = convert(EventTraceData, S_ev)
Tr

Let's add some phases to see how conversion works:

In [ ]:
Tr.pha[1]["P"] = SeisPha(amp = 1.0e6, d = 40075.0, tt = 1344.0, unc = 1.5)
Tr.pha[2] = randPhaseCat()
S_ev2 = convert(SeisData, Tr)
S_ev == S_ev2                    # true unless S_ev.x contains NaNs

...but since the phase catalog wasn't retained by the `SeisData` object `S_ev2`,

In [ ]:
Tr2 = convert(EventTraceData, S_ev2)
Tr == Tr2